In [1]:
# Should run following command for correct connection
# export MLFLOW_S3_ENDPOINT_URL=http://ip:9000
# export AWS_SECRET_ACCESS_KEY=BudxuXerLsZdP5J7Y4dsifCWCCVtvVdieUwMn927
# export AWS_ACCESS_KEY_ID=H0aH9fCBmpmAFIK7GOCF
!pip install mlflow mlserverimport mlflow
from mlserver.types import InferenceResponse, InferenceRequest
from mlserver.codecs.string import StringRequestCodec
from requests.models import Response
import json
import numpy as np

In [5]:
ip = "158.160.52.53"
mlflow.set_tracking_uri(f"http://{ip}:5000")
 
class MyModel(mlflow.pyfunc.PythonModel):
    def predict(self, context, request_input):
        print(f"Полученный predict запрос: {request_input}, тип: {type(request_input)}")
        model_input = self.decode_request(request_input)
        print(f"Результат decode_request для передачи в модель: {model_input}, тип: {type(model_input)}")
        model_output = self.my_custom_function(model_input)
        print(f"Результат работы модели (my_custom_function): {model_output}, тип: {type(model_output)}")
        inference_response = self.encode_response_model(model_output)
        print(f"Обработанный ответ модели для передачи mlserver: {request_input}, тип: {type(request_input)}")
        return inference_response
 
    def my_custom_function(self, model_input: dict) -> dict:
        model_output = {"result": model_input["a"] * 2}
        return model_output
 
    def decode_request(self, model_input: InferenceRequest) -> dict:
        raw_json = StringRequestCodec.decode_request(model_input)
        print(f"raw_json = {raw_json}, тип raw_json: {type(raw_json)}")
        print(f"raw_json[0] = {raw_json[0]}, тип raw_json[0] : {type(raw_json[0])}")
        _input = json.loads(raw_json[0])
        return _input
        
    def encode_response_model(self, model_output: dict) -> dict:
        _output = json.dumps(model_output)
        print(f"_output = {_output}, тип _output: {type(_output)}")
        _output_np = np.array(_output, dtype='object')
        print(f"_output_np = {_output_np}, тип _output_np: {type(_output_np)}")
        inference_response = {"output": _output_np}
        return inference_response
""" 
INFO: needs to give model_name for encode_response, before extract it from request
    def encode_response_model(self, model_output: dict) -> InferenceResponse:
        _output = json.dumps(model_output)
        print(f"_output = {_output}, тип _output: {type(_output)}")
        _output_dict = json.loads(_output)
        print(f"_output_dict = {_output_dict}, тип _output_dict: {type(_output_dict)}")
        inference_response = StringRequestCodec.encode_response(_output_dict)
        print(f"inference_response = {inference_response}, тип inference_response: {type(inference_response)}")
        return inference_response
"""

' \nINFO: needs to give model_name for encode_response, before extract it from request\n    def encode_response_model(self, model_output: dict) -> InferenceResponse:\n        _output = json.dumps(model_output)\n        print(f"_output = {_output}, тип _output: {type(_output)}")\n        _output_dict = json.loads(_output)\n        print(f"_output_dict = {_output_dict}, тип _output_dict: {type(_output_dict)}")\n        inference_response = StringRequestCodec.encode_response(_output_dict)\n        print(f"inference_response = {inference_response}, тип inference_response: {type(inference_response)}")\n        return inference_response\n'

In [6]:
my_model = MyModel()
model_path = "work_model1"
reg_model_name = "work_model1"
with mlflow.start_run(run_name="run_work_model1_v1") as run:
    model_path = f"{model_path}"
    mlflow.pyfunc.save_model(path=model_path, python_model=my_model)
    mlflow.pyfunc.log_model(artifact_path=model_path, python_model=my_model, registered_model_name=reg_model_name)

Successfully registered model 'work_model1'.
2024/02/19 13:16:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: work_model1, version 1
Created version '1' of model 'work_model1'.
